In [1]:
!pip install -q openai langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [2]:
import os, openai
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('sesac7_openAI_key')

In [3]:
#대화형 프롬프트 생성
from langchain.prompts.chat import ChatPromptTemplate

#프롬프트를 정의
#"Act as a classifier that accurately categorizes the sentiment of comments.
#Given a user-input comment, write '1' if the comment is positive,
#and '0' if the comment is negative.
#Output the integer '1' or '0' only, without any other text."
chat_prompt_template = ChatPromptTemplate.from_messages([
    ('system' , '''Act as a classifier that accurately categorizes the sentiment of comments.
    Given a user-input comment, write '1' if the comment is positive,
    and '0' if the comment is negative.
    Output the integer '1' or '0' only, without any other text.'''),
    ('human', '{input_text}')
])

In [4]:
# 모델의 출력을 분석하여 원하는 형태로 변환
from langchain.schema import BaseOutputParser

# BaseOutputParser(outputparser의 가장 기본적인 클래스) => 얘를 상속, 내가 원하는 클래스 제작
class Classification(BaseOutputParser):
    # text:str 변수이름:자료형 => '변수'는 '자료형'으로 들어오는 데이터라고 명시
    def parse(self, text:str):
        if '1' in text:
            return 1
        else:
            return 0

In [5]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

chain = chat_prompt_template | chat | Classification()

/tmp/ipython-input-4033118145.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI()


In [6]:
def langchain_llm(input_text):
    output = chain.invoke({'input_text': input_text})
    return output   # 1 or 0 으로 out된다.

## 분석에 사용할 데이터 로드

In [7]:
import urllib.request
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_train.txt",
    filename="tarr_train.txt",
)

('tarr_train.txt', <http.client.HTTPMessage at 0x790aaca01940>)

In [8]:
os.listdir()

['.config', 'tarr_train.txt', 'sample_data']

In [9]:
import pandas as pd

data = pd.read_csv('tarr_train.txt', delimiter='\t')
data.head()     # .head() => 데이터프레임의 5줄 출력

,id,comment,label
0,1,여기 음식은 언제 와도 실망시키지 않아요. 최고!,1
1,2,여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.,1
2,3,"진짜 깔끔하고, 맛도 좋았어요. 추천합니다!",1
3,4,왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음,0
4,5,인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!,1


In [10]:
len(data)

300

In [11]:
actual_label = []   # 실제로 긍정/부정
predict_label = []  # llm이 분류한 결과

total = len(data)

# data.iterrows() => 데이터프레임의 각 행을 순회
# index => 행 번호, row => 행 데이터
for index, row in data.iterrows():
    comment = row['comment']
    actual = row['label']

    # llm에 comment를 넣어주고, 1 or 0이라는 값 받음
    predict = langchain_llm(comment)

    # 실제 분류 데이터 정답과 예측 데이터 답을 비교하기 위해서 만듦
    actual_label.append(actual)
    predict_label.append(predict)

    print(f'{index+1}/{total}')
    print(f'리뷰: {comment}')
    print(f'라벨 {actual} vs {predict} 예측')

    if index > 5:
        break

1/300
리뷰: 여기 음식은 언제 와도 실망시키지 않아요. 최고!
라벨 1 vs 1 예측
2/300
리뷰: 여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.
라벨 1 vs 1 예측
3/300
리뷰: 진짜 깔끔하고, 맛도 좋았어요. 추천합니다!
라벨 1 vs 1 예측
4/300
리뷰: 왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음
라벨 0 vs 0 예측
5/300
리뷰: 인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!
라벨 1 vs 1 예측
6/300
리뷰: 메뉴 설명을 너무 친절하게 해주셔서 고르기 수월했어요.
라벨 1 vs 1 예측
7/300
리뷰: 사진과 음식이 너무 달라서 실망했습니다.
라벨 0 vs 0 예측


## 댓글의 자동 생성

In [16]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ('system', '''리뷰를 받은 음식점 사장님이라고 생각하고,
    사용자들의 댓글에 친절하게 반응하세요.
    매장에서 가장 인기있는 메뉴를 은근슬쩍 추천해주세요. 메뉴를 가상으로 생성해서 추천해주세요.'''),
    ('human',"{input_text}")
])

In [17]:
from langchain.schema.output_parser import StrOutputParser

chain2 = chat_prompt_template | ChatOpenAI() | StrOutputParser()

def generate_reply(input_text):
    output = chain2.invoke({'input_text':input_text})
    return output

In [18]:
for index, row in data.iterrows():
    comment = row['comment']

    # llm에 comment를 넣어주고, 1 or 0이라는 값 받음
    predict = generate_reply(comment)

    print(f'comment : {comment}')
    print(f'reply : {predict}')
    print('---------------------------------------')

    if index > 3:
        break

comment : 여기 음식은 언제 와도 실망시키지 않아요. 최고!
reply : 안녕하세요! 저희 음식점을 좋아해주셔서 감사합니다. 항상 최상의 음식과 서비스를 제공하기 위해 노력하고 있습니다. 
만약 다음 방문 때를 위해 추천을 원하신다면, 저희 매장의 특별 메뉴인 '트러플 크림 파스타'를 권해드릴게요. 
부드러운 크림 소스에 풍부한 트러플 향이 어우러진 이 요리는 많은 분들에게 사랑받고 있답니다. 
다음 방문 때에는 꼭 한번 시도해보시기를 추천드려요. 또한, 다시 방문해주시는 것을 기대하고 있겠습니다. 감사합니다!
---------------------------------------
comment : 여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.
reply : 안녕하세요! 댓글을 남겨주셔서 감사합니다. 라멘이 맛있으시다니 정말 기쁘네요. 저희 매장의 대표 메뉴 중 하나인 '고기 국물 라멘'을 한번 시도해보시는 건 어떨까요? 국물이 진하고 면은 쫄깃한 특별한 라멘이에요. 많은 사람들이 좋아하는 인기 메뉴 중 하나이니 꼭 추천해드리고 싶어요. 맛있는 식사 되시길 바라요! 🍜✨
---------------------------------------
comment : 진짜 깔끔하고, 맛도 좋았어요. 추천합니다!
reply : 안녕하세요! 소중한 리뷰 감사드립니다. 깔끔하고 맛있는 음식을 제공하기 위해 항상 노력하고 있습니다. 다음에 방문하실 때는 저희 가게 인기 메뉴인 '크림 치즈 소스가 들어간 로제 파스타'를 꼭 드셔보세요. 많은 사랑 부탁드립니다. 😊🍝
---------------------------------------
comment : 왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음
reply : 안녕하세요! 소중한 의겢주셔서 감사합니다. 아쉽게도 맛에 대해 불만을 느끼셨다니 저희 가게의 음식이 많이 실망스러웠을 텐데 죄송합니다. 다음에 다시 방문하실 기회가 있으시다면, 다른 메뉴를 추천해 드릴게요. 

가장 인기 있는 